In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
from sklearn.metrics import roc_auc_score, precision_score, confusion_matrix

In [3]:
random.seed(36)
np.random.seed(36)

### **Heart Disease Dataset**
Heart disease is also known as Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year which is about 32% of all deaths globally. CVDs are a group of disorders of the heart and blood vessels and include coronary heart disease, cerebrovascular disease, rheumatic heart disease, and other conditions. Four out of 5CVD deaths are due to heart attacks and strokes, and one-third of these deaths occur prematurely in people under 70 years of age.

In [4]:
df = pd.read_csv("./statlog-heart-dataset.csv")
df.head()

,age,sex,chest,pressure,cholestoral,bloodsugar,electrocardiographic,heartRate,exInducedAngina,oldpeak,slope,numMajorVessels,thal,class
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,yes
1,57,1,2,124,261,0,0,141,0,0.3,1,0,7,yes
2,56,1,3,130,256,1,2,142,1,0.6,2,1,6,yes
3,59,1,4,110,239,0,2,142,1,1.2,2,1,7,yes
4,60,1,4,140,293,0,2,170,0,1.2,2,2,7,yes


In [5]:
df["target"] = df["class"].apply(lambda x: int(x == "yes"))
df = df.drop(["class"], axis=1)
df.head()

,age,sex,chest,pressure,cholestoral,bloodsugar,electrocardiographic,heartRate,exInducedAngina,oldpeak,slope,numMajorVessels,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
2,56,1,3,130,256,1,2,142,1,0.6,2,1,6,1
3,59,1,4,110,239,0,2,142,1,1.2,2,1,7,1
4,60,1,4,140,293,0,2,170,0,1.2,2,2,7,1


In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [7]:
def eval_clf(clf, X_test, y_test):
  y_pred = clf.predict(X_test)
  y_score = clf.predict_proba(X_test)[:, 1]
  y_pred = clf.predict(X_test)

  roc_auc = roc_auc_score(y_test, y_score)
  precision = precision_score(y_test, y_pred)

  print(f"ROC-AUC = {roc_auc}\nPrecision = {precision}")

In [8]:
def compute_rates(clf, X_test, y_test):
  y_pred = clf.predict(X_test)
  cm = confusion_matrix(y_test, y_pred)
  TN, FP, FN, TP = cm.ravel()

  print(f"TPR = {TP}/({TP}+{FN}) = {TP/(TP+FN)}\nFPR = {FP}/({FP}+{TN}) = {TP/(TP+FN)}")

### [**Decision Tree**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_train, y_train)
eval_clf(clf, X_test, y_test)

ROC-AUC = 0.9114035087719299
Precision = 0.8695652173913043


### **True and False Positive Rates**

**True Positive Rate (TPR):** Also known as sensitivity, recall, or hit rate.
- Formula: `TPR = TP / (TP + FN)`
- Where:
  - `TP` represents True Positives.
  - `FN` represents False Negatives.
- Description: Measures the proportion of actual positives that are correctly identified. A higher TPR indicates a better ability of the model to recognize positive instances.

**False Positive Rate (FPR):**
- Formula: `FPR = FP / (FP + TN)`
- Where:
  - `FP` represents False Positives.
  - `TN` represents True Negatives.
- Description: Measures the proportion of actual negatives that are incorrectly identified as positives. A lower FPR indicates a better ability of the model to avoid falsely classifying negatives as positives.

In [10]:
mask_1 = X_test['sex'] == 1
mask_0 = X_test['sex'] == 0

X_test_1 = X_test[mask_1]
y_test_1 = y_test[mask_1]
X_test_0 = X_test[mask_0]
y_test_0 = y_test[mask_0]

In [11]:
print(f"Woman sample: {len(y_test_0)}\nMan sample: {len(y_test_1)}")

Woman sample: 20
Man sample: 48


In [12]:
compute_rates(clf, X_test_1, y_test_1)

TPR = 17/(17+10) = 0.6296296296296297
FPR = 2/(2+19) = 0.6296296296296297


In [13]:
compute_rates(clf, X_test_0, y_test_0)

TPR = 3/(3+0) = 1.0
FPR = 1/(1+16) = 1.0


### **TPR and FPR analysis**

**0. Women preditions**
- TPR: 100% of the actual positives were correctly classified.
- FPR: 22.22% of the actual negative were incorrectly classified.

**1. Men preditions**

- TPR: 71.43% of the actual positives were correctly classified.
- FPR: 15% of actual negative were incorrectly classified.

### **Analysis of Fairness**
- TPR Disparity: More accurate in identifying heart disease in women (TPR = 1.0) compared to men (TPR = 0.7143).

- FPR Disparity: Higher FPR for women (0.2222) than for men (0.1500). This suggests the model is more likely to incorrectly diagnose heart disease in women who do not actually have it.

### **Overall**

- This could indicate a bias where the model is more sensitive to detecting heart disease in women.